In [34]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from datetime import datetime
import matplotlib.colors 
import matplotlib.dates as mdates
import folium
from folium.plugins import HeatMap
import geopandas as gpd
from folium import plugins
import matplotlib.colors
from shapely.geometry import mapping


# Ignore all warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

---

### 1. Data Import and Extract

- Basic data type conversion and checking
- Get only 2022 Feb to 2025 Feb data (3 yr)
- Get only data from dept = 'DPW - Street Maintenance'



In [35]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one level from notebooks/
data_path = os.path.join(project_root, 'data', 'raw', '311_data.csv')
df= pd.read_csv(data_path)

In [36]:
df.columns

Index(['_id', 'group_id', 'num_requests', 'parent_closed', 'status_name',
       'status_code', 'dept', 'request_type_name', 'request_type_id',
       'create_date_et', 'create_date_utc', 'last_action_et',
       'last_action_utc', 'closed_date_et', 'closed_date_utc', 'origin',
       'street', 'cross_street', 'street_id', 'cross_street_id', 'city',
       'neighborhood', 'census_tract', 'council_district', 'ward',
       'police_zone', 'latitude', 'longitude', 'geo_accuracy'],
      dtype='object')

Data Extraction (get only 2022 to 2025 data)

In [37]:
# Convert the date column to datetime if it's not already
date_columns = ['create_date_et', 'create_date_utc', 'last_action_et', 
                'last_action_utc', 'closed_date_et', 'closed_date_utc']

for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Get earliest and latest dates
print("Earliest date:", df['create_date_et'].min())
print("Latest date:", df['create_date_et'].max())

# Get a basic summary of the date range
print("\nDate range summary:")
print(df['create_date_et'].describe())

Earliest date: 2015-04-20 07:37:00
Latest date: 2025-03-10 10:28:00

Date range summary:
count                           815417
mean     2020-07-21 23:49:28.321485056
min                2015-04-20 07:37:00
25%                2018-03-16 10:22:00
50%                2020-09-02 20:16:00
75%                2023-01-11 10:40:00
max                2025-03-10 10:28:00
Name: create_date_et, dtype: object


In [38]:
# Filter data from 2022 to 2025
df_r3y = df[(df['create_date_et'] > '2022-02-28') & (df['create_date_et'] <= '2025-03-31')]

# Show basic info about the filtered dataset
print("311 data for department DPW - Street Maintenance")
print("Number of records for recent 3 years:", len(df_r3y))


# First ensure create_date_et is datetime type
df_r3y['create_date_et'] = pd.to_datetime(df_r3y['create_date_et'])
# Create a date-only column for daily grouping
df_r3y['date'] = df_r3y['create_date_et'].dt.date

# Get daily records count
daily_counts = df_r3y.groupby('date').size()

# Calculate average records per day
print("Average records per day:", daily_counts.mean())
print("Median records per day:", daily_counts.median())
print("\nDaily records summary:")
print(daily_counts.describe())

# Get the busiest and quietest days
print("\nBusiest day:")
print(f"Date: {daily_counts.idxmax()}")
print(f"Number of records: {daily_counts.max()}")

print("\nQuietest day:")
print(f"Date: {daily_counts.idxmin()}")
print(f"Number of records: {daily_counts.min()}")

311 data for department DPW - Street Maintenance
Number of records for recent 3 years: 285374
Average records per day: 265.4641860465116
Median records per day: 291.0

Daily records summary:
count    1075.000000
mean      265.464186
std       156.173289
min         1.000000
25%        92.500000
50%       291.000000
75%       374.500000
max       798.000000
dtype: float64

Busiest day:
Date: 2025-01-21
Number of records: 798

Quietest day:
Date: 2025-03-04
Number of records: 1


In [40]:
# processed_path = os.path.join(project_root, 'data', 'processed', '311_data_r3y.csv')
# df_r3y.to_csv(processed_path, index=False)

Check Dept and Request_type_name for data, and get 'DPW - Street Mainenance' data only

In [41]:
df_r3y['request_type_name'].value_counts()

request_type_name
Weeds/Debris                        28473
Missed Refuse Pick Up               20943
Potholes                            11661
Missed Recycling Pick Up            11004
Refuse Violations                   10914
                                    ...  
Sidewalk, Damaged/DO NOT USE            1
Building or Property Maintenance        1
Other                                   1
Roadway Obstructions                    1
No Dumping Sign (DO NOT USE)            1
Name: count, Length: 324, dtype: int64

In [42]:
df_r3y['dept'].value_counts()

dept
DPW - Refuse                         56899
Permits, Licenses and Inspections    52270
DPW - Street Maintenance             40606
Police - Zones 1-6                   23293
DOMI - Permits                       11562
                                     ...  
Scoobi                                  23
City Clerks Office                       6
Urban Redevelopment Authority            4
Lamar Advertising                        4
PA. American Water - Dist.               2
Name: count, Length: 61, dtype: int64

In [43]:
df_r3y[df_r3y['dept'] == 'DPW - Refuse']['request_type_name'].value_counts()

request_type_name
Missed Refuse Pick Up                   20931
Missed Recycling Pick Up                11003
Refuse Violations                       10889
Blue Bin Containers                      4876
Early Set Out                            3763
Illegal Dumping                          1052
Commercial Refuse/Dumpsters               980
Bulk Pick Up                              767
Late Set Out                              718
Recycling Questions                       560
Recycling Violation                       504
Premature Requests                        403
Leaves, Grass or Other Yard Debris        128
Special Pick Up                           126
Litter, Public Property                   111
Refuse Violations (DO NOT USE)             33
Homeless                                   12
Weeds/Debris                                9
Litter Can, Public                          9
Early Trash Set Out (DO NOT USE)            6
Early Trash Set Out                         4
Litter          

In [44]:
df_r3y[df_r3y['dept'] == 'DPW - Street Maintenance']['request_type_name'].value_counts()

request_type_name
Potholes                    11649
Litter, Public Property      6497
Snow/Ice removal             6320
Overgrowth                   3371
Street Cleaning/Sweeping     3295
                            ...  
Flooding                        1
Prostitution                    1
Boat/Trailer on Street          1
HACP                            1
Brick or Block Repair           1
Name: count, Length: 66, dtype: int64

In [45]:
df_r3y[df_r3y['dept'].fillna('').str.startswith('DPW')]['request_type_name'].value_counts().head(20)

request_type_name
Missed Refuse Pick Up          20939
Potholes                       11649
Missed Recycling Pick Up       11003
Refuse Violations              10909
Litter, Public Property         6609
Snow/Ice removal                6320
Blue Bin Containers             4878
Early Set Out                   3765
Overgrowth                      3371
Street Cleaning/Sweeping        3295
Illegal Dumping                 2462
Pruning (city tree)             2372
Litter Can, Public              1616
Tree Removal                    1314
Tree Fallen Across Road         1118
Commercial Refuse/Dumpsters      982
Dead tree (Public property)      947
Park Shelter                     930
Drainage/Leak                    929
Public Right of Way              922
Name: count, dtype: int64

In [46]:
# Get DPW data
dpw_data = df_r3y[df_r3y['dept'].fillna('').str.startswith('DPW')]

# Create a more straightforward summary
dpw_summary = pd.DataFrame()

# For each DPW department
for dept in dpw_data['dept'].unique():
    # Get top 5 requests for this department
    dept_requests = dpw_data[dpw_data['dept'] == dept]['request_type_name'].value_counts().head(5)
    
    # Create a dictionary for the row
    row_dict = {'Department': [dept]}
    
    # Add request types and counts, handling cases with fewer than 5 types
    for i in range(5):
        if i < len(dept_requests):
            row_dict[f'Top_{i+1}_Request'] = [dept_requests.index[i]]
            row_dict[f'Top_{i+1}_Count'] = [dept_requests.values[i]]
        else:
            row_dict[f'Top_{i+1}_Request'] = [None]
            row_dict[f'Top_{i+1}_Count'] = [None]
    
    # Create department row and append to summary
    dept_row = pd.DataFrame(row_dict)
    dpw_summary = pd.concat([dpw_summary, dept_row])

# Reset index for clean display
dpw_summary = dpw_summary.reset_index(drop=True)

# Display the summary
print("\nDPW Departments and their Top 5 Request Types:")
display(dpw_summary)

# Optional: Show total requests handled by each de partment
total_requests = dpw_data.groupby('dept').size().sort_values(ascending=False)
print("\nTotal requests handled by each DPW department:")
print(total_requests)




DPW Departments and their Top 5 Request Types:


,Department,Top_1_Request,Top_1_Count,Top_2_Request,Top_2_Count,Top_3_Request,Top_3_Count,Top_4_Request,Top_4_Count,Top_5_Request,Top_5_Count
0,DPW - Administration,Park Shelter,930,Public Works Department,840,Thank you - DPW,575,Barricades,1,Bridge Maintenance,1
1,DPW - Facilities,Water/Drinking Fountains,685,City Facility,675,Water Pressure,1,None,None,None,None
2,DPW - Refuse,Missed Refuse Pick Up,20931,Missed Recycling Pick Up,11003,Refuse Violations,10889,Blue Bin Containers,4876,Early Set Out,3763
3,DPW - Forestry Division,Pruning (city tree),2370,Tree Removal,1314,Dead tree (Public property),946,Storm Damage,813,Dead Tree (3TB),395
4,DPW - Construction Division,Bike Lane Bollard,612,Board Up (City-owned property only),151,Dead Tree (3TB),22,Bicycle/Pedestrian Concerns,11,Traffic Calming,3
5,DPW - Street Maintenance,Potholes,11649,"Litter, Public Property",6497,Snow/Ice removal,6320,Overgrowth,3371,Street Cleaning/Sweeping,3295
6,DPW - Park Maintenance,Illegal Dumping,1405,Playground,664,Field,516,Lights,515,Litter,425
7,DPW - 2nd Division,Graffiti in Right of Way,585,"Graffiti, Removal",129,"Signs, Advertising or Political",1,None,None,None,None
8,DPW - Environmental Enforcement,Courtesy Pick Up,209,Plastic Bag Ban Compliance,97,Illegal Dumping,1,Refuse Violations,1,None,None



Total requests handled by each DPW department:
dept
DPW - Refuse                       56899
DPW - Street Maintenance           40606
DPW - Forestry Division             6951
DPW - Park Maintenance              4549
DPW - Administration                2348
DPW - Facilities                    1361
DPW - Construction Division          801
DPW - 2nd Division                   715
DPW - Environmental Enforcement      308
dtype: int64


In [47]:
df_filtered = df_r3y[df_r3y['dept']== 'DPW - Street Maintenance']

### 2. Data Validation
- **Date validation** : verify time frame and date conversion
  - Check create_date_et distribution
  - Confirm data completeness for each year
  - Verify logical relationships (e.g., closed_date after create_date)

- Validate **Data Consistency**
  - Check for duplicate request IDs
  - Verify we only have data from DPW - Street Maintenance dept
  - Detect inconsistent request_type_id mappings and standardize request types
  - check for missing and invalid coordianate, verify neighborhood and district information

In [48]:
df_filtered.head()

,_id,group_id,num_requests,parent_closed,status_name,status_code,dept,request_type_name,request_type_id,create_date_et,...,city,neighborhood,census_tract,council_district,ward,police_zone,latitude,longitude,geo_accuracy,date
64,65,755693,1,f,in progress,3,DPW - Street Maintenance,"Litter, Public Property",831,2023-07-08 11:38:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2023-07-08
85,86,880597,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-28 08:23:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-28
176,177,882900,1,t,closed,1,DPW - Street Maintenance,Street Cleaning/Sweeping,317,2024-09-05 09:41:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-09-05
453,454,873480,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-06 11:35:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-06
668,669,738943,1,f,open,0,DPW - Street Maintenance,"Litter, Public Property",831,2023-05-22 10:12:00,...,Pittsburgh,NaN,4.200398e+10,6.0,NaN,NaN,40.446306,-80.027041,EXACT,2023-05-22


In [49]:
## Display basic dataset information (shape, columns, data types)
print(f"Dataset shape: {df_filtered.shape}")
print(f"Dataset columns: {df_filtered.columns}")
df_filtered.info()



Dataset shape: (40606, 30)
Dataset columns: Index(['_id', 'group_id', 'num_requests', 'parent_closed', 'status_name',
       'status_code', 'dept', 'request_type_name', 'request_type_id',
       'create_date_et', 'create_date_utc', 'last_action_et',
       'last_action_utc', 'closed_date_et', 'closed_date_utc', 'origin',
       'street', 'cross_street', 'street_id', 'cross_street_id', 'city',
       'neighborhood', 'census_tract', 'council_district', 'ward',
       'police_zone', 'latitude', 'longitude', 'geo_accuracy', 'date'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 40606 entries, 64 to 815412
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   _id                40606 non-null  int64         
 1   group_id           40606 non-null  int64         
 2   num_requests       40606 non-null  int64         
 3   parent_closed      40606 non-null  object        
 4   st

In [50]:
## Data Preprocessing - datetime conversion
df_filtered['create_date_et'] = pd.to_datetime(df_filtered['create_date_et'])
df_filtered['last_action_et'] = pd.to_datetime(df_filtered['last_action_et'])
df_filtered['closed_date_et'] = pd.to_datetime(df_filtered['closed_date_et'])


print("Date Validation Results:")

# 1. Check create_date range
min_date = df_filtered['create_date_et'].min()
max_date = df_filtered['create_date_et'].max()
expected_start = pd.Timestamp('2022-01-01')
expected_end = pd.Timestamp('2025-12-31')

if min_date >= expected_start and max_date <= expected_end:
    print(f"\n✓ Date range within expected bounds (2022-2025)")
else:
    print(f"\n✗ Date range outside expected bounds (2022-2025)")
print(f"   Range: {min_date} to {max_date}")

# 2. Confirm data completeness for each year
year_counts = df_filtered['create_date_et'].dt.year.value_counts().sort_index()
print("\nData distribution by year:")
for year, count in year_counts.items():
    print(f"   {year}: {count} records")

# Check if all years in range have data
expected_years = range(2022, 2026)  # 2022-2025
missing_years = [year for year in expected_years if year not in year_counts.index]
if not missing_years:
    print(f"\n✓ Data available for all expected years (2022-2025)")
else:
    print(f"\n✗ Missing data for years: {', '.join(map(str, missing_years))}")

# 3. Check for logical date order (closed date should be after create date)
invalid_dates = df_filtered[df_filtered['closed_date_et'] < df_filtered['create_date_et']]
if len(invalid_dates) == 0:
    print("\n✓ Date order validation passed")
    print("   All closed dates occur after their create dates")
else:
    print("\n✗ Date order validation failed")
    print(f"   Found {len(invalid_dates)} records with closed_date before create_date")
    print("   Sample of invalid records:")
    print("   " + str(invalid_dates[['_id', 'create_date_et', 'closed_date_et']].head(5)))

# 4. Check for missing dates
missing_create = df_filtered['create_date_et'].isna().sum()
missing_closed = df_filtered['closed_date_et'].isna().sum()

if missing_create == 0:
    print(f"\n✓ No missing create dates")
else:
    print(f"\n✗ Found missing create dates")
    print(f"   create_date_et: {missing_create} ({missing_create/len(df_filtered):.2%})")

if missing_closed == 0:
    print(f"\n✓ No missing closed dates")
else:
    print(f"\n∆ Found missing closed dates")
    print(f"   closed_date_et: {missing_closed} ({missing_closed/len(df_filtered):.2%})")

# 5. Check for future dates
future_dates = df_filtered[df_filtered['create_date_et'] > pd.Timestamp.now()]
if len(future_dates) == 0:
    print("\n✓ No future creation dates")
else:
    print("\n✗ Found records with future creation dates")
    print(f"   {len(future_dates)} records ({len(future_dates)/len(df_filtered):.2%} of dataset)")

print("\nDate validation complete")



Date Validation Results:

✓ Date range within expected bounds (2022-2025)
   Range: 2022-02-28 00:24:00 to 2025-02-04 14:04:00

Data distribution by year:
   2022: 12688 records
   2023: 11082 records
   2024: 12848 records
   2025: 3988 records

✓ Data available for all expected years (2022-2025)

✓ Date order validation passed
   All closed dates occur after their create dates

✓ No missing create dates

∆ Found missing closed dates
   closed_date_et: 2008 (4.95%)

✓ No future creation dates

Date validation complete


In [51]:
# Data Consistency Validation
print("Data Consistency Validation:")

# 1. Check for duplicate request IDs
duplicate_ids = len(df_filtered) - df_filtered['_id'].nunique()
if duplicate_ids == 0:
    print("\n✓ No duplicate request IDs found")
    print(f"   Unique IDs: {df_filtered['_id'].nunique()}, Total records: {len(df_filtered)}")
else:
    print("\n✗ Duplicate request IDs detected")
    print(f"   Found {duplicate_ids} duplicate records")
    print("   Sample duplicates:")
    print(df_filtered[df_filtered['_id'].duplicated(keep=False)].sort_values('_id').head())

# 2. Verify department data
dept_counts = df_filtered['dept'].value_counts(dropna=False)
print("\nDepartment distribution:")
for dept, count in dept_counts.items():
    print(f"   {dept}: {count} records ({count/len(df_filtered):.2%})")

# Check if we only have DPW - Street Maintenance department
target_dept = 'DPW - Street Maintenance'
if len(dept_counts) == 1 and target_dept in dept_counts:
    print(f"\n✓ All records are from '{target_dept}' department")
else:
    print(f"\n✗ Found records from departments other than '{target_dept}'")
    print(f"   {len(dept_counts)} different departments found")

# 3. Check request_type_id consistency
inconsistent_ids = []
request_type_matches = df_filtered.groupby('request_type_id')['request_type_name'].nunique()
inconsistent_types = request_type_matches[request_type_matches > 1]
if len(inconsistent_types) == 0:
    print("\n✓ All request_type_id values map to a single request_type_name")
else:
    print("\n✗ Inconsistent request type mappings detected")
    print(f"   {len(inconsistent_types)} request_type_id values map to multiple names")
    for type_id in inconsistent_types.index:
        names = df_filtered[df_filtered['request_type_id'] == type_id]['request_type_name'].unique()
        inconsistent_ids.append(type_id)
        print(f"   ID {type_id} maps to: {', '.join(names)}")

# 4. Check for invalid/missing coordinates
invalid_coords = df_filtered[(df_filtered['latitude'].isna()) | (df_filtered['longitude'].isna()) | 
                    (df_filtered['latitude'] == 0) | (df_filtered['longitude'] == 0)]
if len(invalid_coords) == 0:
    print("\n✓ All records have valid coordinates")
else:
    print("\n∆ Found records with missing or invalid coordinates")
    print(f"   {len(invalid_coords)} records ({len(invalid_coords)/len(df_filtered):.2%} of dataset)")

# 5. Validate status_code to status_name mappings. Check for inconsistent mappings
status_mappings = df_filtered.groupby('status_code')['status_name'].nunique()
inconsistent_status = status_mappings[status_mappings > 1]

print(f"\nStatus Code Validation:")
if len(inconsistent_status) == 0:
    print(f"✓ All status_code values map to a single status_name")
else:
    print(f"✗ Found {len(inconsistent_status)} status_code values with inconsistent mappings")
    
    # Display details for inconsistent mappings
    for code in inconsistent_status.index:
        name_counts = df_filtered[df_filtered['status_code'] == code]['status_name'].value_counts()
        dominant_name = name_counts.index[0]
        print(f"\nCode {code}:")
        for name, count in name_counts.items():
            print(f"   '{name}': {count} records{' (DOMINANT)' if name == dominant_name else ''}")

# 6. Standardize if inconsistencies found
if len(inconsistent_status) > 0:
    # Create standardized copy
    df_fixed = df_filtered.copy()
    changes = 0
    
    # For each inconsistent code, standardize to dominant name
    for code in inconsistent_status.index:
        code_mask = df_filtered['status_code'] == code
        name_counts = df_filtered[code_mask]['status_name'].value_counts()
        dominant_name = name_counts.index[0]
        
        # Apply standardization
        to_change = df_filtered[code_mask & (df_filtered['status_name'] != dominant_name)]
        changes += len(to_change)
        df_fixed.loc[code_mask, 'status_name'] = dominant_name
    
    # Print summary
    print(f"\nStandardized {changes} records with inconsistent status names")
    
    # Verify fixes
    status_check = df_fixed.groupby('status_code')['status_name'].nunique()
    if (status_check > 1).any():
        print("✗ Some inconsistencies remain")
    else:
        print("✓ All status_code values now map to exactly one status_name")

print("\nData consistency validation complete")


Data Consistency Validation:

✓ No duplicate request IDs found
   Unique IDs: 40606, Total records: 40606

Department distribution:
   DPW - Street Maintenance: 40606 records (100.00%)

✓ All records are from 'DPW - Street Maintenance' department

✗ Inconsistent request type mappings detected
   3 request_type_id values map to multiple names
   ID 478 maps to: Litter Can, Public, Litter Can
   ID 160741 maps to: Potholes, Potholes (DO NOT USE)
   ID 203058 maps to: Snow/Ice Removal (DO NOT USE), Snow/Ice Removal

∆ Found records with missing or invalid coordinates
   229 records (0.56% of dataset)

Status Code Validation:
✓ All status_code values map to a single status_name

Data consistency validation complete


In [55]:
# Standardize all inconsistent request type mappings
def standardize_request_types(df, inconsistent_ids):

    print(f"Found {len(inconsistent_ids)} request_type_id values with inconsistent mappings")

    # 1. Create a standardized copy of the dataframe
    df_standardized = df.copy()
    total_changes = 0

    # 2. For each inconsistent ID, standardize to the dominant name
    for id_val in inconsistent_ids:
        # Find records with this ID
        id_mask = df['request_type_id'] == id_val
        
        # Get name frequencies and identify the dominant name
        name_counts = df[id_mask]['request_type_name'].value_counts()
        dominant_name = name_counts.index[0]
        
        # Print details
        print(f"\nID {id_val}:")
        for name, count in name_counts.items():
            print(f"   '{name}': {count} records{' (DOMINANT)' if name == dominant_name else ''}")
        
        # Count records that need to be changed
        records_to_change = df[id_mask & (df['request_type_name'] != dominant_name)]
        changes_for_this_id = len(records_to_change)
        total_changes += changes_for_this_id
        
        print(f"   Standardizing {changes_for_this_id} records to '{dominant_name}'")
        
        # Apply the standardization
        df_standardized.loc[id_mask, 'request_type_name'] = dominant_name

    # 3. Print summary
    print(f"\nStandardization summary:")
    print(f"Total records modified: {total_changes} ({total_changes/len(df):.2%})")
    print(f"Records before: {len(df)}")
    print(f"Records after: {len(df_standardized)}")

    # 4. Verify the changes
    print("\nVerification after standardization:")
    request_type_counts_after = df_standardized.groupby('request_type_id')['request_type_name'].nunique()
    inconsistent_ids_after = request_type_counts_after[request_type_counts_after > 1].index.tolist()
    if not inconsistent_ids_after:
        print("✓ All request_type_id values now map to exactly one request_type_name")
    else:
        print(f"✗ Found {len(inconsistent_ids_after)} remaining inconsistencies")

    return df_standardized

    # Replace the original dataframe if all checks pass
    # df = df_standardized.copy()


df_standardized = standardize_request_types(df_filtered, inconsistent_ids)


Found 3 request_type_id values with inconsistent mappings

ID 478:
   'Litter Can, Public': 84 records (DOMINANT)
   'Litter Can': 6 records
   Standardizing 6 records to 'Litter Can, Public'

ID 160741:
   'Potholes (DO NOT USE)': 8 records (DOMINANT)
   'Potholes': 7 records
   Standardizing 7 records to 'Potholes (DO NOT USE)'

ID 203058:
   'Snow/Ice Removal (DO NOT USE)': 7 records (DOMINANT)
   'Snow/Ice Removal': 2 records
   Standardizing 2 records to 'Snow/Ice Removal (DO NOT USE)'

Standardization summary:
Total records modified: 15 (0.04%)
Records before: 40606
Records after: 40606

Verification after standardization:
✓ All request_type_id values now map to exactly one request_type_name


In [57]:
## Print Head Data after Data Validation Process
print(f"Data Shape before dropping date outliers and unused date columns: {df_filtered.shape}")
print(f"Data Shape after dropping date outliers and unused date columns: {df_standardized.shape}")
df_standardized.head()

Data Shape before dropping date outliers and unused date columns: (40606, 30)
Data Shape after dropping date outliers and unused date columns: (40606, 30)


,_id,group_id,num_requests,parent_closed,status_name,status_code,dept,request_type_name,request_type_id,create_date_et,...,city,neighborhood,census_tract,council_district,ward,police_zone,latitude,longitude,geo_accuracy,date
64,65,755693,1,f,in progress,3,DPW - Street Maintenance,"Litter, Public Property",831,2023-07-08 11:38:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2023-07-08
85,86,880597,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-28 08:23:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-28
176,177,882900,1,t,closed,1,DPW - Street Maintenance,Street Cleaning/Sweeping,317,2024-09-05 09:41:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-09-05
453,454,873480,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-06 11:35:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-06
668,669,738943,1,f,open,0,DPW - Street Maintenance,"Litter, Public Property",831,2023-05-22 10:12:00,...,Pittsburgh,NaN,4.200398e+10,6.0,NaN,NaN,40.446306,-80.027041,EXACT,2023-05-22


### Save Processed Data

In [58]:
processed_DPW_path = os.path.join(project_root, 'data', 'processed', 'DPW_data_r3y.csv')
df_standardized.to_csv(processed_DPW_path, index=False)
df_standardized

,_id,group_id,num_requests,parent_closed,status_name,status_code,dept,request_type_name,request_type_id,create_date_et,...,city,neighborhood,census_tract,council_district,ward,police_zone,latitude,longitude,geo_accuracy,date
64,65,755693,1,f,in progress,3,DPW - Street Maintenance,"Litter, Public Property",831,2023-07-08 11:38:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2023-07-08
85,86,880597,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-28 08:23:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-28
176,177,882900,1,t,closed,1,DPW - Street Maintenance,Street Cleaning/Sweeping,317,2024-09-05 09:41:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-09-05
453,454,873480,1,t,closed,1,DPW - Street Maintenance,Trail Maintenance,541,2024-08-06 11:35:00,...,Pittsburgh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NULL_COORDINATES,2024-08-06
668,669,738943,1,f,open,0,DPW - Street Maintenance,"Litter, Public Property",831,2023-05-22 10:12:00,...,Pittsburgh,NaN,4.200398e+10,6.0,NaN,NaN,40.446306,-80.027041,EXACT,2023-05-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815392,815667,925883,1,t,closed,1,DPW - Street Maintenance,Potholes,484,2025-02-01 10:09:00,...,Pittsburgh,South Side Slopes,NaN,3.0,16.0,3.0,40.422040,-79.973212,EXACT,2025-02-01
815395,815670,926050,1,f,open,0,DPW - Street Maintenance,Potholes,484,2025-02-02 16:47:00,...,Pittsburgh,South Side Flats,NaN,3.0,16.0,3.0,40.426190,-79.971110,EXACT,2025-02-02
815399,815674,925871,1,f,open,0,DPW - Street Maintenance,"Litter, Public Property",831,2025-02-01 08:37:00,...,Pittsburgh,Brookline,NaN,4.0,19.0,6.0,40.389058,-80.010414,EXACT,2025-02-01
815403,815678,925981,1,f,open,0,DPW - Street Maintenance,Drainage/Leak,470,2025-02-01 21:08:00,...,Pittsburgh,Brookline,NaN,4.0,19.0,6.0,40.389815,-80.009684,EXACT,2025-02-01
